In [1]:
from pathlib import Path
import pandas as pd
from sklearn import pipeline
import plotly.graph_objects as go

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

input_folder = Path('input')
output_folder = Path('output')

In [2]:
df = pd.read_csv(input_folder / '211022_seqana_data_science_challenge_dataset.csv')
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,5091.0,2011.011589,0.285303,2011.000000,2011.000000,2011.000000,2011.000000,2019.000000
dem_nasa_dem30,5091.0,703.000982,689.395483,-64.000000,178.500000,383.000000,1228.000000,3531.000000
dem_merit,5091.0,702.989512,689.818076,-62.485870,177.082237,382.266815,1228.776917,3521.041748
dem_gmted,5091.0,704.887056,689.329857,-62.000000,179.000000,384.000000,1230.000000,3523.000000
dem_srtm90_v4,5091.0,704.098409,689.650488,-64.000000,179.000000,383.000000,1229.000000,3580.000000
...,...,...,...,...,...,...,...,...
sur_refl_b01_std_sampling_year,5091.0,1065.661213,1005.472648,44.805950,222.691322,639.540645,1741.368234,3867.800669
sur_refl_b02_std_sampling_year,5091.0,1068.488921,618.375897,121.464499,588.573861,924.875167,1392.175076,3193.912846
sur_refl_b03_std_sampling_year,5091.0,1036.451850,1079.210268,25.795410,107.901579,649.356386,1767.075432,4049.046509
sur_refl_b07_std_sampling_year,5091.0,453.205834,197.820808,72.230034,305.694851,415.923707,572.038427,1249.752010


# 1
Exploratory data analysis: show covariation/correlations between the different variables of the dataset

In [3]:
covariance = df.cov()
correlation = df.corr()

fig = go.Figure(data=go.Heatmap(
    z=correlation,
    x=correlation.columns,
    y=correlation.index,
))
fig.update_layout(title='Feature Correlations', width=800, height=800, xaxis= {'visible': False}, yaxis= {'visible': False})
fig.write_html(str(output_folder / 'correlation.html'), include_plotlyjs='cnd', full_html=False)
fig.show()

# 2
Plot the distribution of the soc_stock_t_ha variable and determine the covariates with the
highest correlation with this variable.

In [4]:
target = 'soc_stock_t_ha'

fig = go.Figure(data=go.Histogram(
    x=df[target]
))
fig.update_layout(title='Target Distribution')
fig.write_html(str(output_folder / 'target.html'), include_plotlyjs=False, full_html=False)
fig.show()

print(f'mean: {df[target].mean()}, std: {df[target].std()}')

mean: 69.39901772854756, std: 67.24117738445003


In [5]:
target_corr = correlation[[target]].copy().drop(target)
target_corr['abs_correlation'] = target_corr[target].abs()
target_corr = target_corr.sort_values('abs_correlation', ascending=False)

fig = go.Figure(data=[
    go.Bar(
        x=target_corr.index,
        y=target_corr[target].clip(0),
        name='positive corr'
    ),
    go.Bar(
        x=target_corr.index,
        y=(-target_corr[target]).clip(0),
        name='negative corr'
    )
])
fig.update_layout(title='Correlation with target variable', barmode='stack')
fig.write_html(str(output_folder / 'target_corr.html'), include_plotlyjs=False, full_html=False)
fig.show()

target_corr.head(10)

,soc_stock_t_ha,abs_correlation
soil_grids_soc_5_15,0.518204,0.518204
soil_grids_soc_0_5,0.496673,0.496673
soil_grids_ocd_5_15,0.491073,0.491073
soil_grids_nitrogen_0_5,0.480961,0.480961
soil_olm_soc_b0,0.477840,0.477840
soil_grids_ocd_0_5,0.464519,0.464519
soil_olm_soc_b10,0.455652,0.455652
soil_grids_ocs_0_30,0.449386,0.449386
LST_Day_1km_09_mean,-0.429938,0.429938
soil_grids_soc_15_30,0.424652,0.424652


# 3 - 4 - 5
Build a modelling pipeline that let’s a user select a subset of covariates to serve as
explanatory variables and an algorithm from a preselection of ML models to model
soc_stock_t_ha as the target variable.

The pipeline should split the dataset into a training and test dataset, train a model, use it
to predict on the test data and measure and showcase the accuracy of the results.

The modelling pipeline can be contained in a jupyter notebook or run through a script in
a docker container.

In [6]:
def regressor_pipeline(df, target, features, model):
    """Simple pipeline

    Args:
        df (DataFrame): The full Dataframe.
        target (str): Name of the target.
        features (list): list of the features to use
        model (??): model to use

    Returns:
        pipeline, rmse, mae, r2
    """
    X_train, X_test, y_train, y_test = train_test_split(df.drop(target, axis=1), df[target], random_state=42)

    pipeline = Pipeline([
        ('selector', ColumnTransformer([
            ('selector', 'passthrough', features)
        ], remainder='drop')),
        ('scaler', StandardScaler()),
        ('model', model),
        ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)

    rmse = mean_squared_error(y_test, y_pred, squared=False)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return pipeline, rmse, mae, r2

In [7]:
features = list(target_corr.index[:50])
model = LinearRegression()
_, rmse, mae, r2 = regressor_pipeline(df, target, features, model)
print(f'Linear Regressor\nrmse: {rmse}, mae: {mae}, r2: {r2}\n')


model = RandomForestRegressor(n_estimators=100, random_state=42)
_, rmse, mae, r2 = regressor_pipeline(df, target, features, model)
print(f'Random Forest\nrmse: {rmse}, mae: {mae}, r2: {r2}\n')


Linear Regressor
rmse: 53.9257955012893, mae: 31.834105623920507, r2: 0.36618503054815343

Random Forest
rmse: 52.31042365158424, mae: 31.2781537972939, r2: 0.40358872389742395

